# Extracting GLDAS-2.0

---

should have: 
<br>earthengine-api
<br>geemap

In [137]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import time
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

In [138]:
import ee

ee.Initialize()

### Initialization
Initialization of the area, the point, the time period.

In [139]:
aoi = ee.Geometry.Point(-62.119581801721324,-10.215769049745338)
gldas = (ee.ImageCollection('NASA/GLDAS/V021/NOAH/G025/T3H')
         .filterDate(datetime.datetime(2020, 1, 15), datetime.datetime(2020, 1, 16))
         .filterBounds(aoi))


# gldas = gldas.filterDate(datetime.datetime(2000, 1, 1), datetime.datetime(2020, 1, 16))

#practice ^ only 1 day so i can't mess up too much.

In [140]:
Map = emap.Map(center=[-15,-60], zoom=4)


#### Selecting bands / defining their names in-program

I put all the bands as they are, according to [Earth Explorer](https://explorer.earthengine.google.com/#detail/NASA%2FGLDAS%2FV20%2FNOAH%2FG025%2FT3H).

However, it seems that not all data is used in [Mr. Effrain's code.](https://code.earthengine.google.com/c54330f0b4cbcc19cc24a98f9ea0b531) The excess bands have been commented out.


In [141]:
Albedo_inst = gldas.select('Albedo_inst')
Lwnet_tavg = gldas.select('Lwnet_tavg')
Psurf_f_inst = gldas.select('Psurf_f_inst')
Qair_f_inst = gldas.select('Qair_f_inst')
Qh_tavg = gldas.select('Qh_tavg')
Qle_tavg  = gldas.select('Qle_tavg')
Rainf_f_tavg = gldas.select('Rainf_f_tavg')
RootMoist_inst = gldas.select('RootMoist_inst')
SWdown_f_tavg = gldas.select('SWdown_f_tavg')
Swnet_tavg = gldas.select('Swnet_tavg')
Tair_f_inst = gldas.select('Tair_f_inst')
Wind_f_inst = gldas.select('Wind_f_inst')

In [142]:
# AvgSurfT_inst = gldas.select('AvgSurfT_inst')
# CanopInt_inst = gldas.select('CanopInt_inst')
# ECanop_tavg = gldas.select('ECanop_tavg')
# ESoil_tavg = gldas.select('ESoil_tavg')
# Evap_tavg = gldas.select('Evap_tavg')
# LWdown_f_tavg = gldas.select('LWdown_f_tavg')
# PotEvap_tavg = gldas.select('PotEvap_tavg')
# Qg_tavg = gldas.select('Qg_tavg')
# Qs_acc = gldas.select('Qs_acc')
# Qsb_acc = gldas.select('Qsb_acc')
# Qsm_acc = gldas.select('Qsm_acc')
# Rainf_tavg = gldas.select('Rainf_tavg')
# SWE_inst = gldas.select('SWE_inst')
# SnowDepth_inst = gldas.select('SnowDepth_inst')
# Snowf_tavg = gldas.select('Snowf_tavg')
# Snowf_tavg = gldas.select('Snowf_tavg')
# SoilMoi0_10cm_inst = gldas.select('SoilMoi0_10cm_inst')
# SoilMoi10_40cm_inst = gldas.select('SoilMoi10_40cm_inst')
# SoilMoi40_100cm_inst = gldas.select('SoilMoi40_100cm_inst')
# SoilMoi100_200cm_inst = gldas.select('SoilMoi100_200cm_inst')
# SoilTMP0_10cm_inst = gldas.select('SoilTMP0_10cm_inst')
# SoilTMP10_40cm_inst = gldas.select('SoilTMP10_40cm_inst')
# SoilTMP40_100cm_inst = gldas.select('SoilTMP40_100cm_inst')
# SoilTMP100_200cm_inst = gldas.select('SoilTMP100_200cm_inst')
# Tveg_tavg = gldas.select('Tveg_tavg')


#### Main Algorithm applied
Description here

In [143]:
def addtime(image):
    return image.addBands(image.metadata('system:time_start'))

RootMoist_inst = RootMoist_inst.map(addtime)

In [144]:
# def addDate(image):
#     date = ee.Date(image.get('system:index'))
#     date = date.format("YYYY-mm-dd")
#     return image.set({'Date': date})

# RootMoist_inst = RootMoist_inst.map(addDate)

In [145]:
# def date(image):
#     date = ee.Date(image.get('system:time_start'))
#     return image.set({'Date': date})

# RootMoist_inst = RootMoist_inst.map(date)

In [146]:
# def addTime(image):
#     return image.addBands(image.metadata('system:time_start'))
# RootMoist_inst = RootMoist_inst.map(addTime)

In [147]:
def func(collection):   
    def reduce(image):
        reduced = image.reduceRegions(collection=aoi, reducer=ee.Reducer.mean(), scale=25000)
        image = image.set('value', reduced)
        return reduced
    red_feature = collection.map(reduce)
    return red_feature.flatten()

In [148]:
results = func(RootMoist_inst)
results.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-62.119581801721324, -10.215769049745338]},
   'id': 'A20200115_0000_0',
   'properties': {'RootMoist_inst': 491.3389892578125,
    'system:time_start': 1579046400000}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-62.119581801721324, -10.215769049745338]},
   'id': 'A20200115_0300_0',
   'properties': {'RootMoist_inst': 490.59600830078125,
    'system:time_start': 1579057200000}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-62.119581801721324, -10.215769049745338]},
   'id': 'A20200115_0600_0',
   'properties': {'RootMoist_inst': 490.1700134277344,
    'system:time_start': 1579068000000}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-62.119581801721324, -10.215769049745338]},
   'id': 'A20200115_0900_0',
   'properties': {'RootMoist_inst': 489.51800537109375,
  

#### Exporting CSV Table

In [149]:
foldername = "FNS"
ExprtDescrptn = "RootMoist_inst10"

export_task = ee.batch.Export.table.toDrive(collection=results, folder=foldername, description=ExprtDescrptn, fileFormat='CSV')
export_task_config = {
    'scale': 27750, 'driveFolder': 'FNS', "maxPixels": 1.0E13
}
export_task.start()

# ee.batch.data.startProcessing(export_task, export_task_config)

In [ ]:
# Map.addLayerControl()
# Map

Below: Practice / Tests
<br> dead code

In [ ]:
    
#     def featureTime(feature):
#         date = ee.Date(feature.getNumber("system:time_start"))
# #         date = ee.Date(feature.getNumber("system:start_time"))
#         print(date)
#         return feature.set({'Date': date})
        
        

# # A20200115_1500        

# #         date = datetime.datetime.utcfromtimestamp(date.getInfo()["value"]/1000.0)

#         return feature.set({'Date': date1})

    

In [ ]:
# reduced = image.reduceRegions(collection=aoi, reducer=ee.Reducer.mean(), scale=25000)
#         image = image.set('value', reduced)
#         return image

In [ ]:
# def addinfo(feature):
#         date1 = ee.Date(feature.get("system:time_start"))
#         date2 = ee.Date.format(date=date1, format='YYYY-mm-dd')

# #         date = datetime.datetime.utcfromtimestamp(date.getInfo()["value"]/1000.0)

#         return feature.set({'Date': date2})

In [ ]:
# mergedate = RootMoist_inst.merge(daterootmoist_inst)

In [ ]:
# exports.temporalCollection = function(collection, start, count, interval, units) {
#   // Create a sequence of numbers, one for each time interval.
#   var sequence = ee.List.sequence(0, ee.Number(count).subtract(1));

#   var originalStartDate = ee.Date(start);

#   return ee.ImageCollection(sequence.map(function(i) {
#     // Get the start date of the current sequence.
#     var startDate = originalStartDate.advance(ee.Number(interval).multiply(i), units);

#     // Get the end date of the current sequence.
#     var endDate = originalStartDate.advance(
#       ee.Number(interval).multiply(ee.Number(i).add(1)), units);

#     return collection
#         .filterDate(startDate, endDate)
#         .mosaic()
#         .set('system:time_start', startDate.millis());
#   }));
# }

In [ ]:
#         date_string = str(date)
#         dateformat = datetime.datetime.strptime(date_string, "%Y %m %d %H %M")


In [ ]:
# #need a function (algorithm) that reduces regions

# def reduce(image):
#     reduced = image.reduceRegion(collection=aoi, reducer=ee.Reducer.mean(), scale=25000)
#     image = image.set('value', reduced)
#     return image

# def ident_info(feature):
#     return feature.set('date', image.date().format())


In [ ]:
# ImageCollection.map(algorithm, dropNulls)	Collection
# Image.reduceRegions(collection, reducer, scale, crs, crsTransform, tileScale)	FeatureCollection